In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained VGG16 model (assuming it was saved with custom top layers)
vgg_model = load_model('my_pest_classification_model.h5')

# Set parameters
input_shape = (224, 224, 3)  # EfficientNetB0 default input size
num_classes = 132  # Update with your number of classes
batch_size = 32
epochs = 20
learning_rate = 0.001

# Load EfficientNetB0 (or any other EfficientNet variant) without top layers
efficient_net = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Set EfficientNet layers to be trainable
efficient_net.trainable = True  # Set to False if you want to freeze its layers initially

# Replace VGG16 top layers with EfficientNetB0 architecture
x = efficient_net.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional: adjust units to your preference
predictions = Dense(num_classes, activation='softmax')(x)

# Create a new model by combining EfficientNet base with the new top layers
model = Model(inputs=efficient_net.input, outputs=predictions)

# Compile the new model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load the data
train_generator = train_datagen.flow_from_directory(
    r'D:\pest\train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    r'D:\pest\test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

# Callbacks for saving the model and early stopping
checkpoint = ModelCheckpoint('efficientnet_retrained_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model (starting from EfficientNet layers)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping])

# Save the retrained model
model.save('efficientnet_final_retrained_model.h5')


Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.
Epoch 1/20


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


929/929 ━━━━━━━━━━━━━━━━━━━━ 6301s 6s/step - accuracy: 0.3110 - loss: 3.1026 - val_accuracy: 0.3782 - val_loss: 2.5807
Epoch 2/20
  1/929 ━━━━━━━━━━━━━━━━━━━━ 1:24:05 5s/step - accuracy: 0.4062 - loss: 2.0580

C:\Users\Abhishek\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


929/929 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.4062 - loss: 2.0580 - val_accuracy: 0.4545 - val_loss: 2.4201
Epoch 3/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 6377s 7s/step - accuracy: 0.4992 - loss: 1.9251 - val_accuracy: 0.4929 - val_loss: 1.9990
Epoch 4/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5000 - loss: 2.1059 - val_accuracy: 0.3636 - val_loss: 2.3373
Epoch 5/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 5836s 6s/step - accuracy: 0.5611 - loss: 1.6171 - val_accuracy: 0.5085 - val_loss: 1.9738
Epoch 6/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.5312 - loss: 1.7974 - val_accuracy: 0.6364 - val_loss: 1.4790
Epoch 7/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 4953s 5s/step - accuracy: 0.5999 - loss: 1.4420 - val_accuracy: 0.5148 - val_loss: 1.9453
Epoch 8/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 5s 977us/step - accuracy: 0.4375 - loss: 1.9546 - val_accuracy: 0.5455 - val_loss: 1.9057
Epoch 9/20
929/929 ━━━━━━━━━━━━━━━━━━━━ 3814s 4s/step - accuracy: 0.6357 - loss: 1.3006 - val_accuracy:

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming your test images are in a folder 'test' with subfolders for each class
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values between 0 and 1

# Define the test generator
test_generator = test_datagen.flow_from_directory(
    r'D:\pest\test',         # Path to the test data directory
    target_size=(224, 224),   # Image size expected by the model (same as training)
    batch_size=32,            # Set an appropriate batch size
    class_mode='categorical', # Assuming this is a multi-class classification problem
    shuffle=False             # No need to shuffle test data
)


Found 12747 images belonging to 132 classes.


In [4]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


399/399 - 503s - 1s/step - accuracy: 0.5183 - loss: 2.1341
Test Accuracy: 0.52


In [5]:
# Predict on the test data
predictions = model.predict(test_generator)

# Convert predictions to class labels
predicted_classes = predictions.argmax(axis=1)

# Get true labels from the test generator
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Generate classification report and confusion matrix
print(classification_report(true_classes, predicted_classes, target_names=class_labels))
print(confusion_matrix(true_classes, predicted_classes))


399/399 ━━━━━━━━━━━━━━━━━━━━ 506s 1s/step
                                 precision    recall  f1-score   support

                  Adristyrannus       0.22      0.53      0.31        45
       Aleurocanthus spiniferus       0.70      0.53      0.60        66
                    Ampelophaga       0.39      0.58      0.47       103
    Aphis citricola Vander Goot       0.44      0.15      0.22        47
               Apolygus lucorum       0.48      0.18      0.26        56
           Bactrocera tsuneonis       0.45      0.23      0.30        22
                Beet spot flies       0.43      0.23      0.30        40
                    Black hairy       1.00      0.84      0.91        93
   Brevipoalpus lewisi McGregor       0.50      0.29      0.36         7
             Ceroplastes rubens       0.48      0.47      0.48        32
           Chlumetia transversa       0.18      0.05      0.08        42
          Chrysomphalus aonidum       1.00      0.23      0.37        31
        

C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name and similar pests
def predict_pest(image_path, model, class_labels, top_n=5):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    
    # Get the most likely class
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Get the top N predicted classes with their confidence scores
    top_n_indices = np.argsort(prediction[0])[::-1][:top_n]
    top_n_pests = [(class_labels[i], prediction[0][i]) for i in top_n_indices]
    
    # Main prediction based on confidence threshold
    if confidence > 0.5:  # You can adjust the threshold as needed
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}")
    else:
        print("Sorry, I don't recognize the species.")
    
    # Display the top N similar predictions with their probabilities
    print("\nTop similar pests and their probabilities:")
    for i, (pest_name, prob) in enumerate(top_n_pests, 1):
        print(f"{i}. {pest_name}: {prob*100:.2f}% confidence")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels, top_n=5)  # Get top 5 similar pests


Enter the path to the pest image:  C:\Users\Abhishek\Downloads\krzysztof-niewolny-pb5EBwMUc2w-unsplash.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Sorry, I don't recognize the species.

Top similar pests and their probabilities:
1. Locustoidea: 40.45% confidence
2. Cicadella viridis: 25.25% confidence
3. Xylotrechus: 13.86% confidence
4. Miridae: 6.03% confidence
5. Papilio xuthus: 5.66% confidence


In [12]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name and similar pests
def predict_pest(image_path, model, class_labels, top_n=5):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    
    # Get the most likely class
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Get the top N predicted classes with their confidence scores
    top_n_indices = np.argsort(prediction[0])[::-1][:top_n]
    top_n_pests = [(class_labels[i], prediction[0][i]) for i in top_n_indices]
    
    # Main prediction based on confidence threshold
    if confidence > 0.5:  # You can adjust the threshold as needed
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}")
    else:
        print("Sorry, I don't recognize the species.")
    
    # Display the top N similar predictions with their probabilities
    print("\nTop similar pests and their probabilities:")
    for i, (pest_name, prob) in enumerate(top_n_pests, 1):
        print(f"{i}. {pest_name}: {prob*100:.2f}% confidence")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels, top_n=5)  # Get top 5 similar pests


Enter the path to the pest image:  C:\Users\Abhishek\Desktop\github upload\pest\pest_data_set\pest\pest\alfalfa plant bug\31863.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Pest is found to be: Miridae

Top similar pests and their probabilities:
1. Miridae: 94.18% confidence
2. alfalfa plant bug: 5.72% confidence
3. Apolygus lucorum: 0.06% confidence
4. tarnished plant bug: 0.05% confidence
5. aphids: 0.00% confidence


In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img) 
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name and similar pests
def predict_pest(image_path, model, class_labels, top_n=5):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    
    # Get the most likely class
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Get the top N predicted classes with their confidence scores
    top_n_indices = np.argsort(prediction[0])[::-1][:top_n]
    top_n_pests = [(class_labels[i], prediction[0][i]) for i in top_n_indices]
    
    # Main prediction based on confidence threshold
    if confidence > 0.5:  # You can adjust the threshold as needed
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}with confidence {confidence:.2f}")
    else:
        print("Sorry, I don't recognize the species.")
    
    # Display the top N similar predictions with their probabilities
    print("\nTop similar pests and their probabilities:")
    for i, (pest_name, prob) in enumerate(top_n_pests, 1):
        print(f"{i}. {pest_name}: {prob*100:.2f}% confidence")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels, top_n=5)  # Get top 5 similar pests


Enter the path to the pest image:  C:\Users\Abhishek\Desktop\github upload\pest\pest_data_set\pest\pest\alfalfa plant bug\31863.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Pest is found to be: Miridaewith confidence 0.94

Top similar pests and their probabilities:
1. Miridae: 94.18% confidence
2. alfalfa plant bug: 5.72% confidence
3. Apolygus lucorum: 0.06% confidence
4. tarnished plant bug: 0.05% confidence
5. aphids: 0.00% confidence
